In [ ]:
%matplotlib widget

In [ ]:
# -*- coding: utf-8 -*-
#
#  Copyright 2024 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# Test installation

This notebook shows how to run the quick test scripts that are provided with gVirtualXray's Python package. They can be used to make sure the installation is working well on your system. You may run them directly from this notebook or from the command line prompt.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please install the package using `!pip install gvxr`
</div>

# Aims of this session

1. Create our first X-ray simulation, step-by-step;
2. Save our X-ray image in a file format that preserves the original dynamic range;
3. Visualise the results with 3 different look-up tables;
4. Visualise the 3D environment.

![](../img/02-visualisation.png)

# Main steps

There are 6 main steps to simulate an X-ray image:

1. Create a renderer (OpenGL context)
2. Set the X-ray source
3. Set the Spectrum
4. Set the Detector
5. Set the Sample
6. Compute the corresponding X-ray image.

![](../img/dragon2.jpg)

| What | Value | Unit |
|------|-------|------|
| Beam shape | point | N/A |
| X-ray source position | -40, 0, 0 | cm |
| Spectrum | 1000 photons of 80 | keV |
| Detector position | 10, 0, 0 | cm |
| Detector orientation | 0, 0, -1 | N/A |
| Detector resolution | 640*320 | pixels |
| Pixel spacing | 0.5 * 0.5 | mm |
| Sample | STL file of the Welsh dragon | mm |
| Material composition | Ti90Al6V4 alloy | N/A |
| Material density | 4.43 | g/cm3 |

## Import packages

- `os` to create the output directory if needed
- `matplotlib` to show 2D images
- `tifffile` to write TIFF files
- `gvxr` to simulate X-ray images

In [ ]:
# Import packages
import os
# import numpy as np # Who does not use Numpy?

import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting
# from matplotlib.colors import LogNorm # Look up table
# from matplotlib.colors import PowerNorm # Look up table

font = {'family' : 'serif',
         'size'   : 15
       }
matplotlib.rc('font', **font)

# # Uncomment the line below to use LaTeX fonts
# # matplotlib.rc('text', usetex=True)

from tifffile import imwrite # Write TIFF files

from gvxrPython3 import gvxr # Simulate X-ray images
# from gvxrPython3.utils import saveProjections # Plot the X-ray image in linear, log and power law scales
# from gvxrPython3.utils import compareWithGroundTruth # Plot the ground truth, the test image, and the relative error map in %
# from gvxrPython3.utils import interactPlotPowerLaw # Plot the X-ray image using a Power law look-up table
# from gvxrPython3.utils import visualise # Visualise the 3D environment if k3D is supported
# from gvxrPython3.utils import plotScreenshot # Visualise the 3D environment if Matplotlib is supported

Make sure the output directory exists

In [ ]:
output_path = "../output_data"
if not os.path.exists(output_path):
    os.mkdir(output_path)

## 1. Create an OpenGL context

The first step is to create the simulation environment, known here as "OpenGL context". 
Two backends are currently available:

- `"OPENGL"`: makes use of the windowing ability of your system. It can be used for realtime visualisations. It is available on Windows, GNU/Linux and MacOS computers.
- `"EGL"`: is for offscreen rendering on GNU/Linux computers. That's the option for cloud instances and supercomputers.

In [ ]:
# Create an OpenGL context
print("Create an OpenGL context")
gvxr.createOpenGLContext();

## 2. Set the X-ray source

In [ ]:
# Create a source
print("Set up the beam")
gvxr.setSourcePosition(-40.0,  0.0, 0.0, "cm");
gvxr.usePointSource();
#  For a parallel source, use gvxr.useParallelBeam();

## 3. Set the Spectrum

In [ ]:
# Set its spectrum, here a monochromatic beam
# 1000 photons of 80 keV (i.e. 0.08 MeV) per ray
gvxr.setMonoChromatic(0.08, "MeV", 1000);
# The following is equivalent: gvxr.setMonoChromatic(80, "keV", 1000);

## 4. Set the Detector

In [ ]:
# Set up the detector
print("Set up the detector");
gvxr.setDetectorPosition(10.0, 0.0, 0.0, "cm");
gvxr.setDetectorUpVector(0, 0, -1);
gvxr.setDetectorNumberOfPixels(640, 320);
gvxr.setDetectorPixelSize(0.5, 0.5, "mm");

## 5. Set the Sample

In [ ]:
# Locate the sample STL file from the package directory
path = os.path.dirname(gvxr.__file__)
fname = path + "/welsh-dragon-small.stl"

# Load the sample data
if not os.path.exists(fname):
    raise IOError(fname)

print("Load the mesh data from", fname);
gvxr.loadMeshFile("Dragon", fname, "mm")

print("Move ", "Dragon", " to the centre");
gvxr.moveToCentre("Dragon");

In [ ]:
# Material properties
print("Set ", "Dragon", "'s material");

# Iron (Z number: 26, symbol: Fe)
gvxr.setElement("Dragon", 26)
gvxr.setElement("Dragon", "Fe")

# Liquid water
gvxr.setCompound("Dragon", "H2O")
gvxr.setDensity("Dragon", 1.0, "g/cm3")
gvxr.setDensity("Dragon", 1.0, "g.cm-3")

# Titanium Aluminum Vanadium Alloy
gvxr.setMixture("Dragon", "Ti90Al6V4")
gvxr.setMixture("Dragon", [22, 13, 23], [0.9, 0.06, 0.04])
# gvxr.setMixture("Dragon", ["Ti", "Al", "V"], [0.9, 0.06, 0.04]) # Not yet implemented
gvxr.setDensity("Dragon", 4.43, "g/cm3")
gvxr.setDensity("Dragon", 4.43, "g.cm-3")

## 6. Compute the corresponding X-ray image.

In [ ]:
# Compute an X-ray image
# We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.
print("Compute an X-ray image");
x_ray_image = np.array(gvxr.computeXRayImage()).astype(np.single)

# Update the visualisation window
gvxr.displayScene()

In [ ]:
# Save the X-ray image in a TIFF file and store the data using single-precision floating-point numbers.
gvxr.saveLastXRayImage(os.path.join(output_path, 'raw_x-ray_image-02.tif'))

# The line below will also works
# imwrite('output_data/raw_x-ray_image-02.tif', x_ray_image)

# Save the L-buffer
gvxr.saveLastLBuffer(os.path.join(output_path, 'lbuffer-02.tif'));

In [ ]:
# Display the X-ray image
# using a linear colour scale
if has_mpl:
    plt.figure(figsize=(10, 5))
    plt.title("Image simulated using gVirtualXray\nusing a linear colour scale")
    plt.imshow(x_ray_image, cmap="gray")
    plt.colorbar(orientation='vertical');
    plt.show()

    # using a logarithmic colour scale
    plt.figure(figsize=(10, 5))
    plt.title("Image simulated using gVirtualXray\nusing a logarithmic colour scale")
    plt.imshow(x_ray_image, cmap="gray", norm=LogNorm(vmin=x_ray_image.min(), vmax=x_ray_image.max()))
    plt.colorbar(orientation='vertical');
    plt.show()

    # using a Power-law colour scale (gamma=0.5)
    plt.figure(figsize=(10, 5))
    plt.title("Image simulated using gVirtualXray\nusing a Power-law colour scale ($\gamma=0.5$)")
    plt.imshow(x_ray_image, cmap="gray", norm=PowerNorm(gamma=1./2.))
    plt.colorbar(orientation='vertical');
    plt.show()

    # Display the X-ray image and compare three different lookup tables
    plt.figure(figsize=(17, 7.5))

    plt.suptitle("Image simulated with gVirtualXray visualised", y=0.75)

    plt.subplot(131)
    plt.imshow(x_ray_image, cmap="gray")
    plt.colorbar(orientation='horizontal')
    plt.title("using a linear colour scale")

    plt.subplot(132)
    plt.imshow(x_ray_image, norm=LogNorm(), cmap="gray")
    plt.colorbar(orientation='horizontal')
    plt.title("using a logarithmic colour scale")

    plt.subplot(133)
    plt.imshow(x_ray_image, norm=PowerNorm(gamma=1./2.), cmap="gray")
    plt.colorbar(orientation='horizontal');
    plt.title("using a Power-law colour scale ($\gamma=0.5$)")

    plt.tight_layout()

    plt.savefig(os.path.join(output_path, 'projection-02.pdf'), dpi=600);

In [ ]:
# Change the sample's colour
# By default the object is white, which is not always pretty. Let's change it to purple.
red = 102 / 255
green = 51 / 255
blue = 153 / 255
gvxr.setColour("Dragon", red, green, blue, 1.0)

# This image can be used in a research paper to illustrate the simulation environment, in which case you may want to change the background colour to white with:
gvxr.setWindowBackGroundColour(1.0, 1.0, 1.0)

# Update the visualisation window
gvxr.displayScene()

In [ ]:
# Take the screenshot and save it in a file
if has_mpl:
    screenshot = gvxr.takeScreenshot()
    plt.imsave(os.path.join(output_path, "screenshot-02.png"), np.array(screenshot))

    # or display it using Matplotlib
    plt.figure(figsize=(10, 10))
    plt.imshow(screenshot)
    plt.title("Screenshot of the X-ray simulation environment")
    plt.axis('off');
    plt.show()

In [ ]:
# Interactive visualisation
# The user can rotate the 3D scene and zoom-in and -out in the visualisation window.

# - Keys are:
#     - Q/Escape: to quit the event loop (does not close the window)
#     - B: display/hide the X-ray beam
#     - W: display the polygon meshes in solid or wireframe
#     - N: display the X-ray image in negative or positive
#     - H: display/hide the X-ray detector
# - Mouse interactions:
#     - Zoom in/out: mouse wheel
#     - Rotation: Right mouse button down + move cursor```
gvxr.renderLoop()

# Cleaning up

Once we have finished it is good practice to clean up the OpenGL contexts and windows with the following command.

In [ ]:
gvxr.terminate()